# "Testing out தமிழ் language tokenizer"
> "In this notebook I test out the sentencepiece tokenizers built previously"
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [nlp, language-model, தமிழ்]
- hide: false

In [ ]:
#hide
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

You can find the which I built the tokenizers [here](https://www.kaggle.com/manimaranp/building-a-tokenizer-for-tamil-with-sentencepiece)

# Introduction

This notebook is intended to experiment with different tokenizers built previously, with varying `vocab_size` values like `8000`, `16000`, `20000`, `30000`, So how do we exactly test a **tokenizer**?, this brings us to why do we need a tokenizer in the first place.

In `English` language we can easily find meaningful units of a sentence, by **whitespaces**. But it is not that easy in other languages, Like in `தமிழ்`, Consider the following sentence,

**இந்த தொழிற்சாலை பணப்பற்றாக்குறை முதலான பல்வேறு இடைஞ்சல்களை தாண்டி 17 ஆண்டுகள் கழித்தே செயல்பாட்டுக்கு வந்துள்ளது.**, this loosely translates to something like **The factory has come in to operation after over 17 years of series of disruptions, including lack of cash.**

We need to focus on the compound word `பணப்பற்றாக்குறை` which means `lack of cash`, that compound word is actually a combination of two words `பணம்` and `பற்றாக்குறை` representing `Cash` and `Deficiency/lack of` A tokenizer for `தமிழ்`, should split the words into two as mentioned above.

Since Tokenization is `the process of demarcating and possibly classifying sections of a string of input characters`<sup>[1](https://en.wikipedia.org/wiki/Lexical_analysis#Tokenization)</sup> or in simple words identifying **linguistically meaningful units** <sup>[2](https://stackoverflow.com/questions/17314506/why-do-i-need-a-tokenizer-for-each-language)</sup> for further processing. We need it for a language model which essentially built to understand the language that it is built for.

Since we built the tokenizer in an **unsupervised**<sup>[3](https://github.com/google/sentencepiece#sentencepiece)</sup> way, There are no numerical ways of gauging the efficiency of the tokenization (AFAIK), so we are left with to try and tokenize some random sentences with the model, and check the tokenizer ourselves.

# Sentences for testing tokenization

I have chosen some sentences at random, Check the comments above each for their translation in `English`.

In [1]:
sentences = [
    # Sita is a mischievous girl
    'சீதா ஒரு குறும்பு பெண்',
    # I remember my childhood
    'எனக்கு என் குழந்தைப் பருவம் நினைவிருக்கிறது',
    # India has successfully tested the Agni-5 missile for the fourth time from Abdul Kalam Island (Wheeler Island) in Odisha.
    'இந்தியா அக்னி-5 வகை ஏவுகணையை நான்காவது முறையாக வெற்றிகரமாக ஒடிசாவிலுள்ள அப்துல் கலாம் தீவிலிருந்து (வீலர் தீவு) சோதித்தது.',
    # The European Union's Galileo satellite system is in operation. It is believed to be the world's most accurate high-precision positioning system.
    'ஐரோப்பிய ஒன்றியத்தின் கலிலியோ செயற்கைகோள் அமைப்பு செயல்பாட்டுக்கு வந்துள்ளது. இது உலகின் மிக துல்லியமான செய்மதி இடஞ்சுட்டலாக இருக்கும் என நம்பப்படுகிறது.',
    # The factory has come in to operation after over 17 years of series of disruptions, including lack of cash.
    'இந்த தொழிற்சாலை பணபற்றாக்குறை முதலான பல்வேறு இடைஞ்சல்களை தாண்டி 17 ஆண்டுகள் கழித்தே செயல்பாட்டுக்கு வந்துள்ளது.',
    # Citizens, witnesses and warriors mourn the death of their king. It is up to the department to regret any loss.
    'தம் மன்னன் இறந்ததற்கு குடிமக்களும் சான்றோரும் வீரர்களும் வருந்திப் பாடுவது கையறுநிலை என்னும் துறையாகும். எந்த இழப்பையும் எண்ணி வருந்துவது கையறுநிலைத் துறைக்குரியது.',
    # The Poems from Sangam Tamil Literature portrays the trading feats of early Tamilian,Tamilians traded across seas and other countries
    'சங்கத்தமிழ்க் கவிதைகள் பழந்தமிழர்தம் வணிகச்சிறப்பைப் பறைசாற்றி நிற்கின்றன. தமிழர் கடல்கடந்து, அயல்நாடுகளிலும் வணிகம் செய்தனர் என்ற செய்திகளைச் சங்கப்பாடல்கள்வழி அறிகின்றோம்.',
    # Everyone stood up to call for a national flag at a school event.
    'பள்ளி நிகழ்ச்சி ஒன்றில் தேசியக் கொடி ஏற்றுமாறு அழைக்க அவரும் எழுந்தார் அனைவரும் எழுந்து நின்றனர்',
]

Let's try to tokenize each one of them.

# Begin Experimentation

## Initial setup

In [2]:
import sentencepiece as spm
from pathlib import Path
from IPython.core.display import display, HTML
from string import Template

sp = spm.SentencePieceProcessor()

TOK_PATH = '/kaggle/input/building-a-tokenizer-for-tamil-with-sentencepiece/tokenizer'

MODEL_PATHS = [p for p in Path(TOK_PATH).glob('*.model')]

In [3]:
#hide
# Just checking
MODEL_PATHS[0]

PosixPath('/kaggle/input/building-a-tokenizer-for-tamil-with-sentencepiece/tokenizer/tok_30000_size.model')

In [4]:
#hide
td_text = Template('<td>$text</td>')
tr_text = Template('<tr><td>$name</td> $li_elem</tr>')

# Utility to generate HTML text for a neat display
def tokenize_and_display_results(text):
    model_texts = []
    for model in MODEL_PATHS:
        # Load model
        sp.Load(str(model))
        
        # tokenize
        tok = sp.EncodeAsPieces(text)
        
        # Prepare html with string templates
        word_html = ''.join([td_text.substitute(text=word) for word in tok])
        list_html = tr_text.substitute(name=model.stem, li_elem=word_html)
        model_texts.append(list_html)
    
    return display(HTML('<table>' + ''.join(model_texts) + '</table>'))

## Sentence Tokenization

I will try to comment on the tokenization with a limited knowledge of தமிழ் grammar, I will refer to the model by the vocab size they are built with (ie) 8000, 16000, 20000, 30000

First sentence `சீதா ஒரு குறும்பு பெண்`

In [7]:
tokenize_and_display_results(sentences[0])

tok_30000_size,▁சீதா,▁ஒரு,▁குறும்ப,ு,▁பெண்
tok_20000_size,▁சீதா,▁ஒரு,▁குறும்ப,ு,▁பெண்
tok_8000_size,▁சீதா,▁ஒரு,▁குறு,ம்பு,▁பெண்
tok_16000_size,▁சீதா,▁ஒரு,▁குறும்ப,ு,▁பெண்


`குறும்பு` is actually not a compound word, so I think apart from the model with `8000` size all other models got it right.

Next sentence `எனக்கு என் குழந்தைப் பருவம் நினைவிருக்கிறது`

In [8]:
tokenize_and_display_results(sentences[1])

In this sentence `I remember my childhood` the model with `20000` and `30000` got the tokenization right.
`16000` was just close, because the letter `ப்` does not have a meaning on it's own.

Next sentence `இந்தியா அக்னி-5 வகை ஏவுகணையை நான்காவது முறையாக வெற்றிகரமாக ஒடிசாவிலுள்ள அப்துல் கலாம் தீவிலிருந்து (வீலர் தீவு) சோதித்தது.`

In [9]:
tokenize_and_display_results(sentences[2])

This is interesting, We can probably say that `8000` vocab model is not doing so well. Looking at the others, both the `16000` vocab model and `20000` split `ஒடிசாவிலுள்ள` meaning to refer something `in Odisha` into `ஒடிசா` and `விலுள்ள`, I think the right split is `ஒடிசாவில்` and `உள்ள`, which the `30000` got right.

Next Sentence

`ஐரோப்பிய ஒன்றியத்தின் கலிலியோ செயற்கைகோள் அமைப்பு
செயல்பாட்டுக்கு வந்துள்ளது. இது உலகின் மிக துல்லியமான
செய்மதி இடஞ்சுட்டலாக இருக்கும் என நம்பப்படுகிறது.`

In [10]:
tokenize_and_display_results(sentences[3])

All of them may have got it wrong with `Galileo` which is split into `கலி`(Gali) and `லியோ`(leo), (Maybe I am wrong on this part) and in GPS `இடஞ்சுட்டலாக` may be called as `இடம்சுட்டல்`(can be loosely translated to Location Pointer) should have been split like `இடம்` and `சுட்டல்` and `ஆக`, Maybe I am expecting too much.

# Conclusion

I will stop here for this blogpost, You can try the other sentences if you want by forking the notebook [here](https://www.kaggle.com/manimaranp/testing-sentencepiece-tokenizer-for-tamil)

Please comment if you think something is wrong somewhere, share it if you have found this interesting.